In [1]:
!nvidia-smi

Fri Mar  1 17:54:10 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:D8:00.0 Off |                  Off |
| N/A   27C    P0    23W / 150W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer

from huggingface_hub import notebook_login
from datasets import Dataset
#imdb = load_dataset("parquet", data_files={'train': 'imdb/plain_text/train-00000-of-00001.parquet', 'test': 'imdb/plain_text/train-00000-of-00001.parquet'})
import pandas as pd
import transformers
import torch
# Load CSV file with pandas
df = pd.read_csv('imdb/IMDB Dataset.csv', nrows=15000)

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


FileNotFoundError: [Errno 2] No such file or directory: 'imdb/IMDB Dataset.csv'

In [3]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_name_or_path = "Llama-2-7b-Chat-GPTQ"
model_basename = "gptq_model-4bit-128g"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# Move the model to evaluation mode to disable dropout, batchnorm etc., which are not needed during inference
model.eval()


CUDA extension not installed.
CUDA extension not installed.
WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
INFO - The layer lm_head is not quantized.
The safetensors archive passed at Llama-2-7b-Chat-GPTQ/gptq_model-4bit-128g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.
Skipping module injection for FusedLlamaMLPForQuantizedModel as curre

LlamaGPTQForCausalLM(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096, padding_idx=0)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): FusedLlamaAttentionForQuantizedModel(
            (qkv_proj): QuantLinear()
            (o_proj): QuantLinear()
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (act_fn): SiLU()
            (down_proj): QuantLinear()
            (gate_proj): QuantLinear()
            (up_proj): QuantLinear()
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attention_layernorm): LlamaRMSNorm()
        )
      )
      (norm): LlamaRMSNorm()
    )
    (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
  )
)

In [4]:
dataset[0]

{'review': "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is du

In [8]:
print("\n\n*** Generate:")

inputs = tokenizer.encode("What are you doing? give me the long answer", return_tensors="pt").to("cuda:0")
outputs = model.generate(input_ids=inputs)
print(tokenizer.decode(outputs[0]))



*** Generate:
<s> What are you doing? give me the long answer.

You are asking me to provide


In [6]:
outputs

tensor([[    1,  1724,   526,   366,  2599, 29973,  2367,   592,   278,  1472,
          1234, 29889,    13,    13,  3492,   526,  6721,   592,   304,  3867]],
       device='cuda:0')

In [33]:
inputs

tensor([[    1,  1724,   526,   366,  2599, 29973]], device='cuda:0')

# Toy Example : text-generation

In [7]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
)

prompt = 'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n'
sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=2,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

print(f"Prompt: {prompt}")

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

Prompt: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Answer: Yes, here are some TV shows that are similar to "Breaking Bad" and "Band of Brothers":

* "The Sopranos" - This HBO series follows the life of Tony Soprano, a New Jersey mob boss, as he navigates the criminal underworld and deals with personal and family issues.
* "The Wire" - This HBO series explores the drug trade in Baltimore from multiple perspectives, including law enforcement, drug dealers, and politicians.
* "Sons of Anarchy" - This FX series follows the lives of a motorcycle club in California as they engage in illegal activities and deal with internal conflicts.
* "Narcos" - This Netflix series tells the story of the rise and fall of Colomb


In [5]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
)

prompt = 'I liked "Parasite" and "Okja". Do you have any recommendations of other shows I might like?\n'
sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=2,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

print(f"Prompt: {prompt}")

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

Prompt: I liked "Parasite" and "Okja". Do you have any recommendations of other shows I might like?

Result: I liked "Parasite" and "Okja". Do you have any recommendations of other shows I might like?

Comment: Sure! If you enjoyed the social commentary and satire of "Parasite" and "Okja", here are some other shows you might enjoy:

1. "Black Mirror" - This anthology series explores the dark side of technology and its impact on society. Each episode is a standalone story with a different cast and setting.
2. "The Handmaid's Tale" - Based on Margaret Atwood's novel, this show is set in a dystopian future where women have lost their rights and are forced into reproductive servitude. It's a powerful exploration of gender, politics, and resistance.
3. "The Good Place" - This comedy-drama follows Eleanor Shellstrode as she navigates the afterlife, trying to


# Toy Example : Cross-entropy loss

In [8]:
prompt = 'I liked "Parasite" and "Okja". Do you have any recommendations of other shows I might like?\n'
input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

model.eval()  # Set the model to evaluation mode

generated_ids = input_ids
cross_entropy_losses = []

with torch.no_grad():  # Disable gradient calculation for inference
    for _ in range(100):  # Assuming a max_length of 200
        outputs = model(input_ids=generated_ids)
        logits = outputs.logits
        
        # Get the logits for the last generated token
        next_token_logits = logits[:, -1, :]
        
        # Sample the next token using top_k sampling
        #filtered_logits = torch.topk(next_token_logits, k=2).values
        filtered_logits=next_token_logits
        probabilities = torch.nn.functional.softmax(filtered_logits, dim=-1)
        next_token = torch.multinomial(probabilities, num_samples=1)
        next_token = next_token.squeeze(0)
        
        # Calculate cross-entropy loss for the generated token
        loss = torch.nn.functional.cross_entropy(filtered_logits, next_token)
        cross_entropy_losses.append(loss.item())

        # Break the loop if EOS token is generated
        if next_token == tokenizer.eos_token_id:
            break

        # Prepare the next input_ids
        generated_ids = torch.cat((generated_ids, next_token.unsqueeze(0)), dim=1)

# Decode the generated ids to text
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(f"Result: {generated_text}")
print(f"Cross-Entropy Losses: {cross_entropy_losses}")

Result: I liked "Parasite" and "Okja". Do you have any recommendations of other shows I might like?

I don't know how to explain it but the way the shows are structured, they are like adult cartoons. They have a similar tone and visual style to them. 
They're usually a bit more cerebral than your average show, and they often deal with complex social and political issues.  
Cross-Entropy Losses: [0.13167010247707367, 0.5086362957954407, 4.912713527679443, 0.01646256633102894, 6.556489552167477e-06, 2.4043514728546143, 2.2355923652648926, 0.03308727219700813, 1.2264177799224854, 0.2166377305984497, 1.5856866836547852, 2.09274959564209, 1.351503849029541, 0.6622921228408813, 5.07388973236084, 0.8909730911254883, 1.308072805404663, 0.00013290952483657748, 1.0819073915481567, 1.783233880996704, 1.4104498624801636, 2.1874465942382812, 7.550106048583984, 2.407466173171997, 0.0009342834819108248, 1.07287787614041e-05, 1.0373163223266602, 0.48168712854385376, 0.26487264037132263, 0.515329957008

In [87]:
del loss

In [109]:
model

LlamaGPTQForCausalLM(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096, padding_idx=0)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): FusedLlamaAttentionForQuantizedModel(
            (qkv_proj): QuantLinear()
            (o_proj): QuantLinear()
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (act_fn): SiLU()
            (down_proj): QuantLinear()
            (gate_proj): QuantLinear()
            (up_proj): QuantLinear()
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attention_layernorm): LlamaRMSNorm()
        )
      )
      (norm): LlamaRMSNorm()
    )
    (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
  )
)

In [139]:
dir(layer.self_attn), layer.self_attn.num_heads, layer.self_attn.head_dim

(['T_destination',
  '__annotations__',
  '__call__',
  '__class__',
  '__delattr__',
  '__dict__',
  '__dir__',
  '__doc__',
  '__eq__',
  '__format__',
  '__ge__',
  '__getattr__',
  '__getattribute__',
  '__gt__',
  '__hash__',
  '__init__',
  '__init_subclass__',
  '__le__',
  '__lt__',
  '__module__',
  '__ne__',
  '__new__',
  '__reduce__',
  '__reduce_ex__',
  '__repr__',
  '__setattr__',
  '__setstate__',
  '__sizeof__',
  '__str__',
  '__subclasshook__',
  '__weakref__',
  '_apply',
  '_backward_hooks',
  '_backward_pre_hooks',
  '_buffers',
  '_call_impl',
  '_forward_hooks',
  '_forward_hooks_with_kwargs',
  '_forward_pre_hooks',
  '_forward_pre_hooks_with_kwargs',
  '_get_backward_hooks',
  '_get_backward_pre_hooks',
  '_get_name',
  '_is_full_backward_hook',
  '_load_from_state_dict',
  '_load_state_dict_post_hooks',
  '_load_state_dict_pre_hooks',
  '_maybe_warn_non_full_backward_hook',
  '_modules',
  '_named_members',
  '_non_persistent_buffers_set',
  '_parameters',
  

In [5]:
activations={}

def get_activation(name):
    def hook(model, input, output):
        # Assuming output is a tuple where the first element contains attention outputs
        attention_output = output[0]  # Adjust this based on the actual output structure
        # Extract activations for the specified head (layer 2, head 4)
        activations[name] = attention_output  # Adjust indices as necessary
    return hook

In [29]:
get_activation

<function __main__.get_activation(name)>

In [15]:
layer.self_attn.qkv_proj.qweight.shape

torch.Size([512, 12288])

In [14]:
!nvidia-smi

Tue Feb 20 20:21:30 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:D8:00.0 Off |                  Off |
| N/A   31C    P0    42W / 150W |   5154MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
layer_idx=0

model.eval()
layer = model.model.model.layers[1]  # Adjust index for layer 2
hook_handles = []

handle = layer.self_attn.qkv_proj.register_forward_hook(get_activation('layer_2'))
hook_handles.append(handle)

# Example input to the model
input_ids = tokenizer.encode("Example input text here Example input input input input input text here Example input text here", return_tensors="pt").to("cuda")

# Perform a forward pass (ensure the input is appropriate for your model)
output = model(input_ids=input_ids)
attention_activations = activations['layer_2'].cpu()


In [18]:
12288/3
n_head=32
n_layer=32


In [15]:
attention_activations

torch.Size([17, 12288])

In [23]:
xq=attention_activations[:,n_head*n_layer*0:n_head*n_layer*1]
xk=attention_activations[:,n_head*n_layer*1:n_head*n_layer*2]
xv=attention_activations[:,n_head*n_layer*2:n_head*n_layer*3]


In [22]:
k

tensor([[-1.2002,  1.6768,  1.0400,  ..., -0.9189, -0.3677, -0.6768],
        [-0.5869,  0.6377,  0.5747,  ..., -1.0449, -0.4924,  0.6240],
        [-0.1613,  0.3076,  0.2163,  ..., -0.6523, -1.1553,  0.8262],
        ...,
        [-0.2495,  0.3125,  0.4607,  ..., -0.6577, -1.2197,  0.8521],
        [-0.1841,  0.2634,  0.4104,  ..., -0.9355, -0.7974, -1.2432],
        [-0.1995,  0.3721,  0.5264,  ..., -0.7510, -0.9424,  0.0424]],
       dtype=torch.float16)